# Counting threshold exceedance events
## Cal-Adapt Analytics Engine: threshold tools

This notebook calculates exceedance events (time periods above or below a certain threshold) at different time scales using the `app.explore.threshold()` tool, using different criteria describing the extreme event severity and duration (for example, how many 3-day periods per year contain at least five hours of temperatures above 100 degrees F?). For any given location using hourly data, we can calculate and display how frequencies of extreme events will change under different global emissions scenarios and across model simulations. This tool can be used for applications such as:
- Examining how the frequency of cold snaps or heat waves of a user-defined temperature (e.g. above 95 degF), duration (e.g. 3 hours or more), and period of time (e.g. for 6 days in a row) will change annually or seasonally
- Exploring changes in the frequency of extreme precipitation events (for example, periods of low to no precipitation or high precipitation within a season or year) under different global emissions scenarios

## Step 0: setup
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
import panel as pn
pn.extension()

import climakitae as ck
from climakitae import threshold_tools
import xarray as xr

Additionally, get set up to make the computing go faster, by executing the following cell. It may take a minute or two to spin up!

In [ ]:
from dask_gateway import GatewayCluster
cluster = GatewayCluster()
cluster.adapt(minimum=0, maximum=16)
client = cluster.get_client()
cluster

## Step 1: Explore threshold exceedance events and plot

Below, use the `app.explore.threshold()` GUI to explore extreme events. First, select the variable you are interested in characterizing extreme events of, and the units of that variable. You can also select an area of interest for exploring extreme events.

The exceedance calculator can then be used to measure the frequency of exceedance events based on four different parameters: threshold to capture events above or below (e.g. temperatures above 80 degF, temperatures below 30 degF), length of time of the event (e.g. temperatures above a threshold for three hours), whether to count frequency of events seasonally or annually, and duration of extreme event occurrences (for example, to measure multi-day heat events, set this to the number of days you want to consider a heat event). 

Once you select the parameters of the event of interest, the plot on the right will show how frequencies of the defined extreme event change through the year 2100 across different GCM simulations, and changes can be explored for each global emissions scenario using the dropdown menu below the plot. Smoothing can optionally be applied to the time series as well.

**Note** the GUI below can be reloaded to reflect new selections using the 'Reload data' button. Static plots can be saved to export to your local computer by using the save button in the lower right of the plot window.

In [ ]:
app = ck.Application()

In [ ]:
app.explore.thresholds()

Next, if we want to export the time series, we need to grab that data first. 

In [ ]:
selected_data = app.retrieve()

In [ ]:
threshold = 300
selected_transform_data = threshold_tools.get_exceedance_count(selected_data, threshold_value = threshold)

We can confirm that this is the correct transform by plotting the data and comparing to the panel above. 

In [ ]:
selected_transform_data.hvplot.line(x = "time")

## Step 2: Export

To export, first pick a format from the dropdown menu.

- We recommend NetCDF, which will work with any number of variables and dimensions in your dataset
- CSV works best up to 2-dimensional data (e.g., lon x lat), and will be compressed and exported with a separate metadata file
- GeoTIFF is not possible as the time series data does not retain a spatial component

In [ ]:
app.export_as()

Next, write in the object you wish to export and your desired filename (in single or double quotation marks).

In [ ]:
app.export_dataset(selected_transform_data, 'my_filename')

Lastly, when you are done, close your cluster resources to free them up for the next time you work. 

In [ ]:
cluster.close()